In [9]:
library(readxl)
library(tidyverse)
library(nnet)
library(lmtest)

In [10]:
# Read the Excel file
data <- read_excel("./data/exercice1.xlsx")
# Display the first few rows of the data
head(data)
str(data)

ID,sexe,Region,Age,etat_civil,N_educatif,Membre_famille,Revenu_famille,Revenu_Mensuelle,Redoublant,⋯,Niveau_educ_pere,Niveau_educ_mere,Sign_ecole,Endroit_freq,Ph_famille,Ricule_Prof,Ricule_collegue,Sensation_apres_ecole,Apres_repture_ecole,Violence
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,2,1,3,5,0,0,1,⋯,2,2,1,1,1,0,1,1,1,1
2,0,1,4,2,3,5,0,0,1,⋯,2,1,1,2,2,1,1,1,1,1
3,0,1,4,2,3,11,0,100,1,⋯,2,1,2,1,3,0,0,1,2,0
5,0,1,3,2,3,4,0,0,1,⋯,1,1,1,1,2,1,1,3,3,1
8,0,1,1,1,3,5,1,0,1,⋯,3,3,3,4,4,0,0,3,2,1
9,0,1,4,3,3,4,1,100,0,⋯,2,3,4,1,4,1,1,1,2,0


tibble [91 × 24] (S3: tbl_df/tbl/data.frame)
 $ ID                   : num [1:91] 1 2 3 5 8 9 10 11 12 15 ...
 $ sexe                 : num [1:91] 0 0 0 0 0 0 0 0 0 0 ...
 $ Region               : num [1:91] 1 1 1 1 1 1 1 1 1 1 ...
 $ Age                  : num [1:91] 2 4 4 3 1 4 4 4 4 4 ...
 $ etat_civil           : num [1:91] 1 2 2 2 1 3 3 2 2 2 ...
 $ N_educatif           : num [1:91] 3 3 3 3 3 3 3 3 3 3 ...
 $ Membre_famille       : num [1:91] 5 5 11 4 5 4 7 8 9 10 ...
 $ Revenu_famille       : num [1:91] 0 0 0 0 1 1 0 0 0 0 ...
 $ Revenu_Mensuelle     : num [1:91] 0 0 100 0 0 100 100 0 0 0 ...
 $ Redoublant           : num [1:91] 1 1 1 1 1 0 1 1 1 1 ...
 $ Prob_Enseignat       : num [1:91] 0 1 0 1 0 0 0 1 0 0 ...
 $ Aide_ecole           : num [1:91] 1 4 4 4 1 4 5 1 5 2 ...
 $ Rq_Comprtement       : num [1:91] 1 4 1 2 1 1 1 1 1 1 ...
 $ Relat_Parent         : num [1:91] 1 1 1 1 1 1 1 1 3 1 ...
 $ Niveau_educ_pere     : num [1:91] 2 2 2 1 3 2 2 1 2 2 ...
 $ Niveau_educ_mere     : nu

In [12]:
model1 <- multinom(Apres_repture_ecole ~ sexe + Age + Region + etat_civil
                   + N_educatif + Membre_famille + Revenu_famille
                   + Revenu_Mensuelle + Redoublant + Prob_Enseignat
                   + Aide_ecole + Rq_Comprtement + Relat_Parent
                   + Niveau_educ_pere + Niveau_educ_mere + Sign_ecole
                   + Endroit_freq + Ph_famille + Ricule_Prof
                   + Ricule_collegue, data = data)
summary(model1)

# weights:  88 (63 variable)


initial  value 124.766493 
iter  10 value 78.622797
iter  20 value 63.239268
iter  30 value 55.258599
iter  40 value 49.846960
iter  50 value 47.101855
iter  60 value 43.719936
iter  70 value 43.481027
iter  80 value 43.465618
iter  90 value 43.465149
final  value 43.465144 
converged


Warning message in sqrt(diag(vc)):
"NaNs produced"


Call:
multinom(formula = Apres_repture_ecole ~ sexe + Age + Region + 
    etat_civil + N_educatif + Membre_famille + Revenu_famille + 
    Revenu_Mensuelle + Redoublant + Prob_Enseignat + Aide_ecole + 
    Rq_Comprtement + Relat_Parent + Niveau_educ_pere + Niveau_educ_mere + 
    Sign_ecole + Endroit_freq + Ph_famille + Ricule_Prof + Ricule_collegue, 
    data = data)

Coefficients:
  (Intercept)      sexe         Age       Region  etat_civil    N_educatif
2   -7.489772 245.87263  -0.4376716   -2.4084838    4.901879 -5.771419e-01
3  -10.469941  47.08085   0.7066992    0.6411367    5.941687  8.525498e-03
4 -611.468711  82.99164 208.3453744 -225.7999524 -128.520080 -1.605677e+02
  Membre_famille Revenu_famille Revenu_Mensuelle   Redoublant Prob_Enseignat
2      0.2134464      0.4469855       0.00356250    0.6462154      1.3745384
3      0.1245541      0.7613484       0.00124591   -0.2674273     -0.3408284
4     23.4909023    -32.9186956       0.42818081 -187.7576202    -38.2412999
  Aide

## Facteurs influençant la rupture scolaire

- **Sexe** :  
  L'impact du sexe est significatif, avec un coefficient de 245.87 pour la catégorie masculine, indiquant que ce facteur a une influence notable. Toutefois, la grande valeur du coefficient pourrait aussi signaler des problèmes de colinéarité avec d'autres variables.

- **Âge** :  
  L'âge a un effet variable, avec un coefficient de -0.44 pour chaque année supplémentaire. Cela suggère que, selon les catégories, l'augmentation de l'âge pourrait réduire légèrement la probabilité de rupture scolaire.

- **Région** :  
  La région semble avoir une influence, mais avec un coefficient négatif de -2.41. Cela pourrait refléter des disparités d'accès à l'éducation selon la zone géographique.

- **État civil et niveau éducatif des parents** :  
  L'état civil a un effet protecteur : un statut marital stable réduit la probabilité de rupture scolaire. Le niveau éducatif des parents (coefficients autour de 0.41 pour l'éducation du père et 0.99 pour la mère) montre qu’un niveau d'éducation plus élevé est lié à un moindre risque de rupture scolaire.

- **Revenu familial et mensuel** :  
  Un revenu familial plus élevé (coefficients de 0.45 et 0.0045 respectivement pour le revenu familial et mensuel) diminue la probabilité d'abandonner l'école, ce qui confirme l'influence des conditions économiques sur l'accès à l'éducation.

- **Redoublement** :  
  Le redoublement a un effet positif, avec un coefficient de 0.65, ce qui indique que les élèves ayant redoublé sont plus susceptibles de quitter l'école.

- **Probabilité d'un enseignant** :  
  La qualité de l'enseignement, représentée par la probabilité d'un enseignant, semble avoir un impact important avec un coefficient de 1.37, suggérant qu'un enseignement de qualité réduit les risques de rupture scolaire.

- **Aide à l'école** :  
  L’existence d’une aide scolaire est protectrice, avec un coefficient de 0.52, réduisant la probabilité d'abandon scolaire.

- **Comportement et relation avec les parents** :  
  Un comportement problématique (coefficient de -1.24) et une mauvaise relation avec les parents (coefficient de -0.67) augmentent significativement le risque de rupture scolaire.

- **Phénomènes familiaux** :  
  Des tensions familiales, indicatives de problèmes familiaux, augmentent aussi le risque d'abandon scolaire, avec un coefficient de 0.82.

- **Railleries des professeurs et collègues** :  
  Le harcèlement scolaire, qu'il vienne des professeurs ou des camarades, est un facteur de risque majeur pour la rupture scolaire, avec des coefficients de -1.19 et 0.82 respectivement.


In [21]:
# Make predictions
predictions <- predict(model1, newdata = data)

# Confusion matrix
confusion_matrix <- table(data$Apres_repture_ecole, predictions)
print(confusion_matrix)

   predictions
     1  2  3  4
  1 29  0  0  0
  2  3 23  5  0
  3  2  5 12  1
  4  0  0  0 10


# Matrice de confusion :

- La première ligne montre que 29 prédictions ont été correctes pour la classe 1 (aucune erreur pour cette classe).
- La deuxième ligne indique que 23 prédictions étaient correctes pour la classe 2, mais 3 ont été incorrectes et ont été classées comme 1, et 5 comme 3.
- La troisième ligne montre que 12 prédictions étaient correctes pour la classe 3, mais 2 ont été incorrectes et ont été classées comme 1, 5 comme 2, et 1 comme 4.
- La quatrième ligne montre que 10 prédictions étaient correctes pour la classe 4, sans erreurs.

In [24]:
# Accuracy calculation
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
print(paste("Accuracy:", accuracy))

[1] "Accuracy: 0.822222222222222"


# Précision globale :

- *Exactitude (Accuracy) de 0.822* : Cela signifie que 82,22 % des prédictions faites par le modèle étaient correctes. C’est un bon indicateur de la performance globale du modèle.